<a href="https://colab.research.google.com/github/vasanththangavel/Web_Map/blob/master/earth_engine_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install earthengine-api
!pip install folium

In [2]:
import ee

In [3]:
## Trigger the authentication flow. You only need to do this once
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=u2ujqp2PEp4g39f5bG5HW4Cmr0U0hBWesL0krZUjxn0&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g4hP3YEi5T85uOK1hM93G841QGjeQ3x18-VpcW7bRaGtoN2ahrU0vI

Successfully saved authorization token.


In [38]:
dem=ee.Image("CGIAR/SRTM90_V4")
point_coords=ee.Geometry.Point([78.3387,11.2485])
elev = dem.sample(point_coords,30)
print("Kolli hills elevation in m",elev.first().get('elevation'))

Kolli hills elevation in m ee.ComputedObject({
  "functionInvocationValue": {
    "functionName": "Element.get",
    "arguments": {
      "object": {
        "functionInvocationValue": {
          "functionName": "Collection.first",
          "arguments": {
            "collection": {
              "functionInvocationValue": {
                "functionName": "Image.sample",
                "arguments": {
                  "image": {
                    "functionInvocationValue": {
                      "functionName": "Image.load",
                      "arguments": {
                        "id": {
                          "constantValue": "CGIAR/SRTM90_V4"
                        }
                      }
                    }
                  },
                  "region": {
                    "functionInvocationValue": {
                      "functionName": "GeometryConstructors.Point",
                      "arguments": {
                        "coordinates": {
              